In [4]:
import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
#import torchvision.transforms as transforms
from torch import nn
from torch import optim
from torch.autograd import Variable
#from torch.utils.data import DataLoader
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import datasets,models,transforms
from torch.utils.data import ConcatDataset

import torchbearer

import os
#from sklearn.model_selection import train_test_split
import PIL.Image as Image
import matplotlib.pyplot as plt
#import warnings

C:\Users\gopik\anaconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\gopik\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device
# fix random seed for reproducibility
seed = 7
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)

In [6]:
import sys
sys.path.append("Topological-CNN/../../../")
import klein
from models import NOL_NOL , KF_NOL, CF_NOL

In [7]:
# Parameters from original report
conv_slices = 2
kernel_size = 3
batch_size = 100
epochs = 5
lr = 1e-5

# Training & Testing

In [8]:
# TODO: is it meant to train and test per batch?

## Training

In [28]:
def train(network, data_tr_loader, lr=1e-5, epochs=5, model_path='model.pth', optimizer_path='optimizer.pth'  ):
    # https://nextjournal.com/gkoehler/pytorch-mnist
    optimizer = optim.Adam(network.parameters(), lr=lr)
    #TODO:
    log_interval = 10
    for epoch in range(epochs):
        network.train()
        for batch_idx, (data, target) in enumerate(data_tr_loader):
            optimizer.zero_grad()
            output = network(data)
            #loss = torch.nn.functional.mse_loss
            #loss = torch.nn.MSELoss(output, target)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                    torch.save(network.state_dict(), model_path)
                    torch.save(optimizer.state_dict(), optimizer_path)
        print('Training complete')

In [45]:
def test(network,data_test_loader, model_path='model.pth', optimizer_path='optimizer.pth'):
    network.eval()
    #network_state_dict = torch.load(model_path)
    #network = network.load_state_dict(network_state_dict)
    test_loss = 0
    correct = 0
    accuracies = []
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_test_loader):
            output = network(data)
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
            accuracy = 100.0 * correct / len(data_test_loader.dataset)
            print(accuracy)
    print('Testing complete')

In [11]:
# def train2(network, data_tr_loader, lr=1e-5, epochs=5):
#     # define the loss function and the optimiser
#     loss_function = nn.torch.nn.MSELoss()
#     optimiser = optim.Adam(model.parameters(),lr=lr)

#     # Construct a trial object with the model, optimiser and loss.
#     # Also specify metrics we wish to compute.
#     trial = torchbearer.Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy'])
    
#     # Provide the data to the trial
#     trial.with_generators(trainloader, test_generator=testloader)

#     # Run  epochs of training
#     trial.run(epochs)

#     # test the performance
#     results = trial.evaluate(data_key=torchbearer.TEST_DATA)
#     print(results)

In [12]:
# def test2(network, data_test_loader):
    

# SVHN & MNIST

## Dataset preparation

In [13]:
def load_MINST(download=False):
    trainset = datasets.MNIST(".", train=True, download=download,
                           transform=transforms.Compose([transforms.ToTensor()]))
    testset = datasets.MNIST(".", train=False, download=download,
                          transform=transforms.Compose([transforms.ToTensor()]))
    return ConcatDataset([trainset, testset]) # 70000 samples
mnist = torch.utils.data.DataLoader(load_MINST(), batch_size=batch_size, shuffle=True)


In [14]:
def load_SVHN(download=False):
    svhn_transform = transforms.Compose([
        transforms.Resize([28, 28]),  # down-resolve from (32 x 32) to (28 x 28)
        transforms.Grayscale(),
        transforms.ToTensor()
    ])

    trainset = datasets.SVHN('SVHN/', split='train', download=download,
                             transform=svhn_transform)
    testset = datasets.SVHN('SVHN/', split='test', download=download,
                            transform=svhn_transform)
    return ConcatDataset([trainset, testset])
svhn = torch.utils.data.DataLoader(load_SVHN(), batch_size=batch_size, shuffle=True)

## SVHN to MNIST

In [37]:
# build the model
nol_nol = NOL_NOL(conv_slices, kernel_size, 10, (28,28))

In [30]:
train(nol_nol, svhn, epochs=1)


Training complete


In [46]:
test(nol_nol,mnist)

tensor(0.0086)
tensor(0.0186)
tensor(0.0357)
tensor(0.0500)
tensor(0.0657)
tensor(0.0914)
tensor(0.1057)
tensor(0.1300)
tensor(0.1500)
tensor(0.1629)
tensor(0.1800)
tensor(0.1957)
tensor(0.2171)
tensor(0.2357)
tensor(0.2571)
tensor(0.2657)
tensor(0.2829)
tensor(0.2971)
tensor(0.3143)
tensor(0.3329)
tensor(0.3486)
tensor(0.3629)
tensor(0.3757)
tensor(0.3914)
tensor(0.4043)
tensor(0.4229)
tensor(0.4471)
tensor(0.4600)
tensor(0.4771)
tensor(0.4971)
tensor(0.5100)
tensor(0.5257)
tensor(0.5386)
tensor(0.5529)
tensor(0.5657)
tensor(0.5829)
tensor(0.6000)
tensor(0.6157)
tensor(0.6243)
tensor(0.6414)
tensor(0.6500)
tensor(0.6657)
tensor(0.6771)
tensor(0.6871)
tensor(0.7057)
tensor(0.7200)
tensor(0.7286)
tensor(0.7443)
tensor(0.7643)
tensor(0.7786)
tensor(0.8000)
tensor(0.8114)
tensor(0.8271)
tensor(0.8400)
tensor(0.8514)
tensor(0.8700)
tensor(0.8943)
tensor(0.9129)
tensor(0.9414)
tensor(0.9571)
tensor(0.9700)
tensor(0.9929)
tensor(1.0100)
tensor(1.0243)
tensor(1.0429)
tensor(1.0557)
tensor(1.0

tensor(8.9614)
tensor(8.9757)
tensor(8.9900)
tensor(8.9971)
tensor(9.0143)
tensor(9.0329)
tensor(9.0457)
tensor(9.0600)
tensor(9.0786)
tensor(9.0929)
tensor(9.1086)
tensor(9.1214)
tensor(9.1371)
tensor(9.1514)
tensor(9.1657)
tensor(9.1814)
tensor(9.1986)
tensor(9.2171)
tensor(9.2371)
tensor(9.2514)
tensor(9.2629)
tensor(9.2786)
tensor(9.2943)
tensor(9.3143)
tensor(9.3400)
tensor(9.3543)
tensor(9.3771)
tensor(9.3971)
tensor(9.4057)
tensor(9.4171)
tensor(9.4343)
tensor(9.4514)
tensor(9.4671)
tensor(9.4786)
tensor(9.4986)
tensor(9.5171)
tensor(9.5357)
tensor(9.5529)
tensor(9.5700)
tensor(9.5800)
tensor(9.5971)
tensor(9.6171)
tensor(9.6357)
tensor(9.6486)
tensor(9.6600)
tensor(9.6757)
tensor(9.6886)
tensor(9.7029)
tensor(9.7200)
tensor(9.7414)
tensor(9.7500)
tensor(9.7686)
tensor(9.7757)
tensor(9.7929)
tensor(9.8043)
tensor(9.8243)
tensor(9.8471)
tensor(9.8671)
tensor(9.8800)
tensor(9.9014)
tensor(9.9157)
tensor(9.9329)
tensor(9.9457)
tensor(9.9586)
tensor(9.9700)
tensor(9.9814)
tensor(10.

##  MNIST to SVHN

In [ ]:
# build the model
model = NOL_NOL(conv_slices, kernel_size, 10, (28,28))

# Kaggle & CIFAR

In [ ]:
from torchvision.datasets import CIFAR10

## CIFAR to KAGGLE

## KAGGLE to CIFAR